In [1]:
import os

# 禁用 JAX 的显存预分配
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

import jax
from evox import Problem, algorithms,jit_class, monitors, workflows
from evox.operators import mutation, crossover, selection
from jax import random
import jax.numpy as jnp

import sys
sys.path.append('../my_es_algorithm')  # 添加路径到系统路径

In [2]:
## 产电机组参数
unit_ids = jnp.arange(1, 14)
alpha = jnp.array([0.00028, 0.00056, 0.00056, 0.00324, 0.00324, 0.00324, 0.00324, 0.00324, 0.00324, 0.00284, 0.00284, 0.00284, 0.00284])
beta = jnp.array([8.1, 8.1, 8.1, 7.74, 7.74, 7.74, 7.74, 7.74, 7.74, 8.6, 8.6, 8.6, 8.6])
gamma = jnp.array([550, 309, 309, 240, 240, 240, 240, 240, 240, 126, 126, 126, 126])
lambda_vals = jnp.array([300, 200, 200, 150, 150, 150, 150, 150, 150, 100, 100, 100, 100])
rho = jnp.array([0.035, 0.042, 0.042, 0.063, 0.063, 0.063, 0.063, 0.063, 0.063, 0.084, 0.084, 0.084, 0.084])
pmin = jnp.array([0, 0, 0, 60, 60, 60, 60, 60, 60, 40, 40, 55, 55])
pmax = jnp.array([680, 360, 360, 180, 180, 180, 180, 180, 180, 120, 120, 120, 120])

# Compiling into a dictionary for structured display or usage
power_units = {
    "Unit": unit_ids,
    "Alpha": alpha,
    "Beta": beta,
    "Gamma": gamma,
    "Lambda": lambda_vals,
    "Rho": rho,
    "Pmin": pmin,
    "Pmax": pmax
}



In [3]:
## 热点联产机组参数
units = jnp.array([14, 15, 16, 17, 18, 19])
a = jnp.array([0.0345, 0.0435, 0.0345, 0.0435, 0.1035, 0.072])
b = jnp.array([14.5, 36, 14.5, 36, 34.5, 20])
c = jnp.array([2650, 1250, 2650, 1250, 2650, 1565])
d = jnp.array([0.03, 0.027, 0.03, 0.027, 0.025, 0.02])
e = jnp.array([4.2, 0.6, 4.2, 0.6, 2.203, 2.34])
f = jnp.array([0.031, 0.011, 0.031, 0.011, 0.051, 0.04])

# Feasible region coordinates for Pc and Hc
feasible_region = {
    14: jnp.array([[98.8, 0], [81, 104.8], [215, 180], [247, 0]]),
    15: jnp.array([[44, 0], [44, 15.9], [40, 75], [110.2, 135.6], [125.8, 32.4], [125.8, 0]]),
    16: jnp.array([[98.8, 0], [81, 104.8], [215, 180], [247, 0]]),
    17: jnp.array([[44, 0], [44, 15.9], [40, 75], [110.2, 135.6], [125.8, 32.4], [125.8, 0]]),
    18: jnp.array([[20, 0], [10, 40], [45, 55], [60, 0]]),
    19: jnp.array([[35, 0], [35, 20], [90, 45], [90, 25], [105, 0]])
}

# Combine into a structured dictionary
chp_units = {
    "Units": units,
    "a": a,
    "b": b,
    "c": c,
    "d": d,
    "e": e,
    "f": f,
    "Feasible_Region": feasible_region
}

In [4]:
## 产热机组参数
# Heat-only unit parameters
heat_units = jnp.array([20, 21, 22, 23, 24])
a_heat = jnp.array([0.038, 0.038, 0.038, 0.052, 0.052])
b_heat = jnp.array([2.0109, 2.0109, 2.0109, 3.0651, 3.0651])
c_heat = jnp.array([950, 950, 950, 480, 480])
Hmin = jnp.array([0, 0, 0, 0, 0])
Hmax = jnp.array([2695.2, 60, 60, 120, 120])

# Combine into a structured dictionary
heat_units = {
    "Units": heat_units,
    "a": a_heat,
    "b": b_heat,
    "c": c_heat,
    "Hmin": Hmin,
    "Hmax": Hmax
}

In [6]:
def adjust_population_random_loop(x, Pd, key, tolerance):  # 多次调整
    n = x.shape[0]  # 获取发电机的数量

    def cond_fun(state):
        # x, sumpop, df, r = state
        x, sumpop = state
        return jnp.abs(sumpop - Pd) >= tolerance

    def body_fun(state):
        x, sumpop = state
        df = sumpop - Pd  # 计算功率偏差
        r = random.randint(key, shape=(), minval=0, maxval=n)  # 生成随机数
        x = x.at[r].set(x[r] - df)  # 使用 .at[].set() 更新特定索引的值
        # x = jnp.clip(x, lb, ub)  # 确保更新后的值不超出上下限
        sumpop = jnp.sum(x)  # 计算更新后所有发电机功率的总和
        return x, sumpop

    state = (x, jnp.sum(x))
    x, _= jax.lax.while_loop(cond_fun, body_fun, state)

    # 新增：将最终结果保留两位小数
    x = jnp.round(x, 2)
    
    return x

In [7]:
## 纯电机组成本
def power_unit_cost(xp):
    return  ((alpha * (xp ** 2) + beta * xp + gamma).sum() + jnp.abs(lambda_vals * jnp.sin(rho * (Pd - xp))).sum())

## 热点联产机组成本
def chp_unit_cost(xp,xh):
    return a * xp**2 + b * xp + c + d * xh + e * xh**2 + f * xp * xh 

## 产热机组成本
def heat_unit_cost(xh):
    return a_heat * xh**2 + b_heat * xh + c_heat

## 总成本
def total_cost(xp,xh):
    return power_unit_cost(xp) + chp_unit_cost(xp,xh) + heat_unit_cost(xh)



In [13]:
import sys
sys.path.append('/home/cyh/evox/my_so_algorithm/')  # 添加包含my_pso.py文件的路径
from my_pso import my_PSO  # 导入my_PSO类

In [14]:
# 初始化PSO优化器
pso = my_PSO(
    lb=lb,
    ub=ub,
    Pd=2350,
    pop_size=100,
    problem_type='chp_eld',
    power_params = power_params
    chp_params=chp_params,
    heat_params=heat_params
)

NameError: name 'lb' is not defined